In [1]:
#install pyreadr on terminal
#sudo pip install pyreadr

In [2]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import os
from termcolor import colored

from sklearn.preprocessing import PolynomialFeatures, OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.metrics import confusion_matrix,precision_recall_curve, roc_curve
from sklearn.metrics import precision_score, recall_score, f1_score,accuracy_score, roc_auc_score

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.pipeline import Pipeline, FeatureUnion

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.linear_model import SGDRegressor, SGDClassifier
from sklearn.linear_model import Ridge,Lasso,ElasticNet
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, BaggingClassifier 
from sklearn.ensemble import BaggingClassifier, GradientBoostingRegressor
from sklearn.svm import SVC,SVR
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA, IncrementalPCA, KernelPCA
from sklearn.cluster import KMeans, MiniBatchKMeans, DBSCAN

Matplotlib is building the font cache; this may take a moment.


In [3]:
import pyreadr

# read in female data
result = pyreadr.read_r('/mnt/ML_HBLUP/NA_RM105_110_115/data/dummyMatrix_female.rds') # also works for RData
# done! 
# result is a dictionary where keys are the name of objects and the values python
# objects. In the case of Rds there is only one object with None as key
femaleData = result[None] # extract the pandas data frame 

# read in male data
result = pyreadr.read_r('/mnt/ML_HBLUP/NA_RM105_110_115/data/dummyMatrix_male.rds') # also works for RData
maleData = result[None] # extract the pandas data frame 


In [4]:
femaleData.head()

,HB1__1067-1,HB1__32843,HB1__64DWA2,HB1__B73,HB1__MANS,HB1__NA,HB1__WDAQ2,HB2__1067-1,HB2__32843,HB2__64DWA2,...,HB17115__7797,HB17115__B73,HB17115__NA,HB17115__RQAA8,HB17116__2FACC,HB17116__7797,HB17116__B73,HB17116__FBMU,HB17116__NA,HB17116__RQAA8
01DHD10,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
01DHD16,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
01DKD2-BGL-T1A1,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
01DKD2-NQR-T1B1,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
01DKD2,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0


In [5]:
maleData.head()

,HB1__01HGI4,HB1__610,HB1__B14,HB1__LH287,HB1__M3AG-3,HB1__NA,HB1__OH43AE1,HB1__PH207,HB2__610,HB2__B14,...,HB17114__OH07,HB17114__PH207,HB17115__LH123,HB17115__NA,HB17115__OH07,HB17115__TA1180,HB17116__LH123,HB17116__NA,HB17116__OH07,HB17116__PH207
LH287,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0
83INI14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
17IFI6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
DILU757,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GEJO564,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0


In [6]:
# add suffix
femaleData.columns += '_f'
maleData.columns += '_m'

In [7]:
maleData.head()

,HB1__01HGI4_m,HB1__610_m,HB1__B14_m,HB1__LH287_m,HB1__M3AG-3_m,HB1__NA_m,HB1__OH43AE1_m,HB1__PH207_m,HB2__610_m,HB2__B14_m,...,HB17114__OH07_m,HB17114__PH207_m,HB17115__LH123_m,HB17115__NA_m,HB17115__OH07_m,HB17115__TA1180_m,HB17116__LH123_m,HB17116__NA_m,HB17116__OH07_m,HB17116__PH207_m
LH287,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0
83INI14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
17IFI6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
DILU757,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GEJO564,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0


In [8]:
femaleData.head()

,HB1__1067-1_f,HB1__32843_f,HB1__64DWA2_f,HB1__B73_f,HB1__MANS_f,HB1__NA_f,HB1__WDAQ2_f,HB2__1067-1_f,HB2__32843_f,HB2__64DWA2_f,...,HB17115__7797_f,HB17115__B73_f,HB17115__NA_f,HB17115__RQAA8_f,HB17116__2FACC_f,HB17116__7797_f,HB17116__B73_f,HB17116__FBMU_f,HB17116__NA_f,HB17116__RQAA8_f
01DHD10,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
01DHD16,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
01DKD2-BGL-T1A1,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
01DKD2-NQR-T1B1,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
01DKD2,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0


In [9]:
allHapFemales = femaleData.index
allHapMales = maleData.index

In [10]:
# read in the train data and test data
trainPheno = pd.read_csv('/mnt/ML_HBLUP/NA_RM105_110_115/data/train_phenoData_NA_Corn_hblup_2015-2020_ALL_UDR_105-110-115.csv')
testPheno = pd.read_csv('/mnt/ML_HBLUP/NA_RM105_110_115/data/test_phenoData_NA_Corn_hblup_2021_ALL_UDR_105-110-115.csv')

In [11]:
trainPheno.head()

,LINE_NAME,FEMALE,MALE,YLD_BE_BLUP,MST_BE_BLUP
0,01DHD10+LH287,01DHD10,LH287,-15.661,-0.219
1,01DHD16+83INI14,01DHD16,83INI14,-41.839,-4.254
2,01DKD2-BGL-T1A1+17IFI6,01DKD2-BGL-T1A1,17IFI6,-30.021,-3.251
3,01DKD2-BGL-T1A1+DILU757,01DKD2-BGL-T1A1,DILU757,8.279,1.769
4,01DKD2-BGL-T1A1+GEJO564,01DKD2-BGL-T1A1,GEJO564,4.171,1.519


In [12]:
testPheno.head()

,Unnamed: 0,LINE_NAME,FEMALE,MALE,YLD_BE_BLUP
0,1,JYDB2078+JULI2041,JYDB2078,JULI2041,3.853463
1,2,FIDA1520+GALV1529,FIDA1520,GALV1529,0.083630
2,3,HIQO1923+HILU1759,HIQO1923,HILU1759,-9.287542
3,4,FIDA1520-TCJ-T1A2+HILU672-WQQ-T1A1,FIDA1520-TCJ-T1A2,HILU672-WQQ-T1A1,0.519811
4,5,DEDD1628+HILV1970,DEDD1628,HILV1970,-1.873619


In [13]:
# filter trainData by selecting hybrids with both female and male are included in haplotypdata
trainPheno = trainPheno[(trainPheno['FEMALE'].isin(allHapFemales)) & trainPheno['MALE'].isin(allHapMales) & trainPheno['YLD_BE_BLUP'].notna()]
testPheno = testPheno[testPheno['FEMALE'].isin(allHapFemales) & testPheno['MALE'].isin(allHapMales) & testPheno['YLD_BE_BLUP'].notna()]

# drop duplicated rows
trainPheno = trainPheno.drop_duplicates()
testPheno = testPheno.drop_duplicates()

In [14]:
print(trainPheno.shape)
print(testPheno.shape)

(33475, 5)
(7307, 5)


In [21]:
# construct haplotype data for test and train data
trainHap = pd.concat([femaleData.loc[trainPheno['FEMALE'],:].reset_index(drop=True),
                      maleData.loc[trainPheno['MALE'],:].reset_index(drop=True)],axis=1)
trainHap = trainHap / 2

testHap = pd.concat([femaleData.loc[testPheno['FEMALE'],:].reset_index(drop=True),
                     maleData.loc[testPheno['MALE'],:].reset_index(drop=True)],axis=1)
testHap = testHap / 2

In [22]:
print(trainHap.shape)
print(testHap.shape)

(33475, 109598)
(7307, 109598)


In [23]:
# splite the train data into train and validation 
seed = 20230510
np.random.seed(seed)
X_train, X_test, y_train, y_test = train_test_split(trainHap, trainPheno['YLD_BE_BLUP'], test_size=0.2, random_state=seed)

In [24]:
# need to install lightgbm from terminal: sudo pip install lightgbm
import lightgbm as lgb

gbm = lgb.LGBMRegressor(num_leaves=20,
                        learning_rate=0.05,
                        n_estimators=900,
                       bagging_fraction =  0.7,
                       feature_fraction = 0.5,
                       objective = "regression")

In [25]:
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='mse',
        callbacks=[lgb.early_stopping(5)])

[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
Training until validation scores don't improve for 5 rounds
Did not meet early stopping. Best iteration is:
[900]	valid_0's l2: 21.0964


LGBMRegressor(bagging_fraction=0.7, feature_fraction=0.5, learning_rate=0.05,
              n_estimators=900, num_leaves=20, objective='regression')

In [26]:
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr

metrics = pd.DataFrame(columns=['Method', 'RMSE', 'corr'])
y_pred = gbm.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
corr, p_value = pearsonr(y_test.ravel(), y_pred.ravel())
metrics_curr_cv = pd.DataFrame(data={'Method': 'LightGBM', 'RMSE': [rmse], 'corr': [corr]})
metrics = pd.concat([metrics, metrics_curr_cv], axis=0)
metrics

,Method,RMSE,corr
0,LightGBM,4.593083,0.931593


# try probability model

In [34]:
# read in female data
result = pyreadr.read_r('/mnt/ML_HBLUP/NA_RM105_110_115/data/astProb_female.rds') # also works for RData
# done! 
# result is a dictionary where keys are the name of objects and the values python
# objects. In the case of Rds there is only one object with None as key
femaleData = result[None] # extract the pandas data frame 

# read in male data
result = pyreadr.read_r('/mnt/ML_HBLUP/NA_RM105_110_115/data/astProb_male.rds') # also works for RData
maleData = result[None] # extract the pandas data frame 

In [35]:
# add suffix
femaleData.columns += '_f'
maleData.columns += '_m'

In [36]:
# construct haplotype data for test and train data
trainHap = pd.concat([femaleData.loc[trainPheno['FEMALE'],:].reset_index(drop=True),
                      maleData.loc[trainPheno['MALE'],:].reset_index(drop=True)],axis=1)
trainHap = trainHap / 2

testHap = pd.concat([femaleData.loc[testPheno['FEMALE'],:].reset_index(drop=True),
                     maleData.loc[testPheno['MALE'],:].reset_index(drop=True)],axis=1)
testHap = testHap / 2

In [37]:
print(trainHap.shape)
print(testHap.shape)

(33475, 109598)
(7307, 109598)


In [38]:
trainHap.head()

,HB1__1067-1_f,HB1__32843_f,HB1__64DWA2_f,HB1__B73_f,HB1__MANS_f,HB1__NA_f,HB1__WDAQ2_f,HB2__1067-1_f,HB2__32843_f,HB2__64DWA2_f,...,HB17114__OH07_m,HB17114__PH207_m,HB17115__LH123_m,HB17115__NA_m,HB17115__OH07_m,HB17115__TA1180_m,HB17116__LH123_m,HB17116__NA_m,HB17116__OH07_m,HB17116__PH207_m
0,0.0,0.0,0.0,0.49665,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00000,0.00000,0.49975,0.0,0.0,0.00000,0.49965,0.0,0.00000
1,0.5,0.0,0.0,0.00000,0.0,0.0,0.0,0.5,0.0,0.0,...,0.0,0.48405,0.00000,0.00000,0.0,0.0,0.00000,0.00000,0.0,0.48175
2,0.0,0.0,0.0,0.00000,0.0,0.5,0.0,0.0,0.0,0.0,...,0.0,0.46920,0.00000,0.00000,0.0,0.0,0.00000,0.00000,0.0,0.46515
3,0.0,0.0,0.0,0.00000,0.0,0.5,0.0,0.0,0.0,0.0,...,0.0,0.00000,0.00000,0.00000,0.0,0.0,0.00000,0.00000,0.0,0.00000
4,0.0,0.0,0.0,0.00000,0.0,0.5,0.0,0.0,0.0,0.0,...,0.0,0.00000,0.49055,0.00000,0.0,0.0,0.48965,0.00000,0.0,0.00000


In [39]:
# splite the train data into train and validation 
seed = 20230510
np.random.seed(seed)
X_train, X_test, y_train, y_test = train_test_split(trainHap, trainPheno['YLD_BE_BLUP'], test_size=0.2, random_state=seed)

In [40]:
del trainHap; del result; del femaleData; del maleData

In [41]:
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='mse',
        callbacks=[lgb.early_stopping(5)])

Training until validation scores don't improve for 5 rounds
Did not meet early stopping. Best iteration is:
[899]	valid_0's l2: 19.8585


LGBMRegressor(bagging_fraction=0.7, feature_fraction=0.5, learning_rate=0.05,
              n_estimators=900, num_leaves=20, objective='regression')

In [42]:
y_pred = gbm.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
corr, p_value = pearsonr(y_test.ravel(), y_pred.ravel())
metrics_curr_cv = pd.DataFrame(data={'Method': 'LightGBM_prob', 'RMSE': [rmse], 'corr': [corr]})
metrics = pd.concat([metrics, metrics_curr_cv], axis=0)
metrics

,Method,RMSE,corr
0,LightGBM,4.593083,0.931593
0,LightGBM_prob,4.456291,0.935599


In [ ]:
# save model

import joblib

joblib.dump(cv_, './marker_model/marker_model_EN.pkl')